In [1]:
%%writefile requirements.txt
langchain
langchain-community
llama-parse
fastembed
chromadb
python-dotenv
langchain-groq
chainlit
fastembed
unstructured[md]

Writing requirements.txt


In [25]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GROQ_API_KEY")
secret_value_1 = user_secrets.get_secret("LLAMA_CLOUD_API_KEY")
secret_value_2 = user_secrets.get_secret("PINECONE_API_KEY")
secret_value_3 = user_secrets.get_secret("token")
secret_value_4 = user_secrets.get_secret("wandb-api-key")

In [23]:
# type(pinecone_)

str

In [3]:
%%capture
import nltk
nltk.download('all')

In [4]:
%%capture
!pip install -r requirements.txt

In [5]:
%%capture
!pip install sentence-transformers
!pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

In [6]:
!pip install --upgrade pinecone 
!pip install protoc-gen-openapiv2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 2.0 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 5.28.2 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 whic

In [7]:
!pip install -U langchain-pinecone pinecone-notebooks

In [8]:
# from llama_parse import LlamaParse

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
#
from groq import Groq
from langchain_groq import ChatGroq
#
import joblib
import os
import nest_asyncio  # noqa: E402
nest_asyncio.apply()

import pinecone
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
# llamaparse_api_key = userdata.get('LLAMA_CLOUD_API_KEY')
# groq_api_key = ""

In [12]:
# Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(
                data,
                glob="*.pdf",
                loader_cls=PyPDFLoader
                )
    documents = loader.load()
    return documents

In [14]:
extracted_data = load_pdf("/kaggle/input/mutual-fund-2023-2024")


In [15]:
len(extracted_data)

2925

In [16]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [17]:
text_chunks = text_split(extracted_data)
# # 

In [18]:
len(text_chunks)

45542

In [19]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



/tmp/ipykernel_30/2780497281.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [24]:
# PINECONE_API_KEY

NameError: name 'PINECONE_API_KEY' is not defined

In [27]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
import pinecone

from pinecone import Pinecone, ServerlessSpec


# Initialize Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'mf-rag'

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [28]:
pc.list_indexes()

[
    {
        "name": "mf-rag",
        "dimension": 384,
        "metric": "cosine",
        "host": "mf-rag-jzrql2f.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    }
]

In [29]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [30]:
from uuid import uuid4
from langchain_core.documents import Document

documents = [Document(page_content=t.page_content) for t in text_chunks]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['30ae1533-8e82-4c28-9258-22e5f6056dd0',
 '47a39239-084e-4341-9cb5-e0a179ff873b',
 'c8a359d9-01c4-44b1-99b4-91b18e11db60',
 '6ebb4c8b-6e98-431f-9417-1564d253872d',
 '65c9ddc7-593b-491b-9fe2-5e78de237dd0',
 '44da2805-18f9-4aac-b023-cfd5b05e2242',
 '006bed7b-6e65-44fb-b329-c360758c7cc0',
 '58b07156-0eb8-4a4d-93ae-b74d6b71a0de',
 '17cf1d7d-7fb3-4fe9-a3c5-d3a161e9061f',
 '05bc5931-0578-4f72-ab1e-d14d3e9601ae',
 '15409cf1-30a9-4209-8fc0-af3b99309cd8',
 '020407ac-f66b-47e6-9c1f-d25194db229f',
 'f750430e-f923-4faa-8e67-aba0b56ac7f0',
 '1d015657-6073-4c11-a3ed-bb89d9490119',
 '7c6f8f0c-3d4e-4b68-a4ac-610bd35a9f03',
 'e2f79e2c-2530-4d41-8910-4bf741f0471d',
 'f59b4a66-d23a-441b-ac5d-4d6ea574fdfa',
 'cd9fa3c8-5ea2-4a1d-b4ad-950fe11255ce',
 'ac63e5e2-a81a-4e81-8cb3-b46ff49eb142',
 'a04a5ba7-0401-4ba7-9e34-7219fd65c42c',
 '3447a3ba-2257-4e10-8380-63c6781a0134',
 'a20325cc-de6a-4c90-94c9-3c7128fcba2b',
 '35088051-1449-472c-b4bb-904c5ee279c2',
 '8381c2c7-3770-43d3-96dc-1cab760eda1d',
 'a959c049-1179-

In [32]:
results = vector_store.similarity_search_with_score(
    "find information related Axis NIFTY 50 ETF FUND", k=3
)
for res, score in results:
    print(f"{res.page_content}")

Axis India Manufacturing Fund 
Axis US Treasury Dynamic Bond ETF Fund of Fund  100/1
500/16
6
100/1 100/1 500/1 6(Formerly known as Axis Triple Advantage Fund)
Tax Saver Fund, Axis Focused 25 Fund, Axis Growth Opportunities Fund, Axis NIFTY 50 Index Fund, Axis NIFTY Next 50 Index Fund, Axis Nifty Smallcap 50 Index Fund, Axis Nifty Midcap 50 Index Fund, Axis NIFTY IT ETF, Axis NIFTY Healthcare ETF, Axis ESG 
Integration Strategy  Fund since 4th Aug, 2023, Axis India Manufacturing Fund since 27th Dec, 2023.
Axis ESG Integration Strategy Fund 2.23% 1.34%
Axis Silver ETF 0.35%
Axis Focused 25 Fund 1.71% 0.76%
Axis Floater Fund 0.57% 0.20%
Axis Greater China Equity Fund of Fund 1.58% 0.62%
Axis Global Equity Alpha Fund of Fund 1.60% 0.74%
Axis Gold ETF 0.56%
Axis Global Innovation Fund of Fund 1.58% 0.58%
Axis Gold Fund 0.50% 0.17%
Axis Growth Opportunities Fund 1.71% 0.57%
Axis NIFTY Healthcare ETF 0.34%


In [33]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)


In [34]:
prompt_template= """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [35]:
PROMPT = PromptTemplate(template=prompt_template, input_variable=["context","question"])
chain_type_kwargs = {"prompt":PROMPT}

In [36]:
chat_model = ChatGroq(temperature=0,
                      model_name="mixtral-8x7b-32768",
                      api_key = "GROQ_API_KEY")

In [37]:
chat_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7eb69e973f40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7eb69e9702b0>, temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [38]:
qa = RetrievalQA.from_chain_type(llm=chat_model,
                               chain_type="stuff",
                               retriever=retriever,
                               return_source_documents=True,
                               chain_type_kwargs=chain_type_kwargs)

In [46]:
response = qa.invoke({"query": "Given a monthly salary of ₹80,000 and savings of ₹5,00,000, should i invest  in  the Motilal Oswal Nifty India Defence Index Fund , based on its performance . Include a breakdown of how much can be invested via SIP and lumpsum. Additionally, predict potential returns based on the fund's past performance and current CAGR trends. Ensure to factor in risk levels and investment horizon."})


In [47]:
response

{'query': "Given a monthly salary of ₹80,000 and savings of ₹5,00,000, should i invest  in  the Motilal Oswal Nifty India Defence Index Fund , based on its performance . Include a breakdown of how much can be invested via SIP and lumpsum. Additionally, predict potential returns based on the fund's past performance and current CAGR trends. Ensure to factor in risk levels and investment horizon.",
 'result': "To make a decision about investing in the Motilal Oswal Nifty India Defence Index Fund, let's analyze the given data and consider your savings and monthly salary.\n\nFirst, let's calculate how much you can invest monthly via a Systematic Investment Plan (SIP):\n\nYour monthly salary is ₹80,000, and you save ₹5,00,000 from it. Assuming you save 25% of your salary, you can invest ₹20,000 per month (25% of ₹80,000) via a SIP.\n\nNow, let's calculate how much you can invest as a lump sum:\n\nYou have ₹5,00,000 in savings. You can invest this amount as a lump sum in the Motilal Oswal Nif

In [48]:
print(response['result'])

To make a decision about investing in the Motilal Oswal Nifty India Defence Index Fund, let's analyze the given data and consider your savings and monthly salary.

First, let's calculate how much you can invest monthly via a Systematic Investment Plan (SIP):

Your monthly salary is ₹80,000, and you save ₹5,00,000 from it. Assuming you save 25% of your salary, you can invest ₹20,000 per month (25% of ₹80,000) via a SIP.

Now, let's calculate how much you can invest as a lump sum:

You have ₹5,00,000 in savings. You can invest this amount as a lump sum in the Motilal Oswal Nifty India Defence Index Fund.

Next, let's predict potential returns based on the fund's past performance and current CAGR trends:

From the given data, the 1-year return of the fund is around 7.06%, and the 15-day return is around 5.97%. Considering the 1-year return, if you invest ₹20,000 per month for a year, your estimated investment would be ₹2,40,000. Based on the 7.06% return, your estimated return after a yea

In [50]:
response = qa.invoke({"query": "Retrieve and list the top-performing ETFs from SBI Mutual Fund over the last year. Include performance metrics such as returns, expense ratios, and other relevant financial indicators. Provide insights into the sectors or asset classes these ETFs are focused on, and compare them to similar funds in the market."})
print(response['result'])

Based on the information provided, I can see only one ETF from SBI Mutual Fund, which is the "SBI NIFTY 10 YR BENCHMARK G-SEC ETF." I don't have real-time or historical data on other SBI Mutual Fund ETFs to compare their performance. However, I can provide some insights into this particular ETF.

SBI NIFTY 10 YR BENCHMARK G-SEC ETF:
- As of the given context, the NAV (Net Asset Value) is 19,155 and the latest price is 40.14.
- Since I don't have the historical data, I cannot provide the returns for the last year. However, the return can be calculated by subtracting the NAV from the latest price and dividing it by the NAV, then multiplying by 100 to get the percentage. Please note that this calculation is based on the provided data, and for an accurate return, you should refer to a reliable financial source.
- This ETF aims to replicate the performance of the Nifty 10 Year Benchmark G-Sec Index, which is a benchmark index for the 10-year government securities market in India.
- As it is

In [52]:
response = qa.invoke({"query": "How did the Nippon India Small Cap Fund perform compared to its benchmark?"})
response['result']

"I don't have the specific performance data for the Nippon India Small Cap Fund. However, the context provided suggests that small cap indices have been performing well, which might lead to a small cap fund outperforming its benchmark. To provide an accurate comparison, the specific benchmark and performance data for the Nippon India Small Cap Fund would be needed."